In [1]:
"""
install
keras, numpy, tensorflow, scipy, Pillow

"""
import os
from google.colab import drive
drive.mount('/content/gdrive')
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import callbacks
import numpy as np
from keras.utils import load_img, img_to_array
import pandas as pd


Mounted at /content/gdrive


In [2]:
os.chdir('gdrive/MyDrive/boddam-masters-project')

In [ ]:
train_data_dir = "sample_data/train"
test_data_dir = "sample_data/test"
img_width, img_height = 200, 200
nb_train_samples = 8
nb_validation_samples = 1
nb_filters1 = 2
nb_filters2 = 4
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 2
batch_size = 2
lr = 0.0001
epochs = 4

In [ ]:
model = Sequential()
model.add(Conv2D(nb_filters1, (conv1_size, conv1_size), padding="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
model.add(Conv2D(nb_filters1, (conv1_size, conv1_size), padding="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=lr),
              metrics=['accuracy'])


In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

train_generator = train_datagen.flow_from_directory(
    directory='sample_data/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 8 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 2 images belonging to 2 classes.


In [ ]:
model.fit_generator(train_generator,
                    #steps_per_epoch = nb_train_samples,
                    epochs = epochs,
                    validation_data = test_generator,
                    #callbacks = cbks,
                    validation_steps = nb_validation_samples)


Epoch 1/4


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


4/4 [==============================] - 2s 261ms/step - loss: 0.8200 - accuracy: 0.5000 - val_loss: 0.7688 - val_accuracy: 0.0000e+00
Epoch 2/4
4/4 [==============================] - 1s 126ms/step - loss: 0.8486 - accuracy: 0.2500 - val_loss: 0.7733 - val_accuracy: 0.5000
Epoch 3/4
4/4 [==============================] - 0s 120ms/step - loss: 0.6897 - accuracy: 0.5000 - val_loss: 0.7664 - val_accuracy: 0.5000
Epoch 4/4
4/4 [==============================] - 1s 124ms/step - loss: 0.7229 - accuracy: 0.5000 - val_loss: 0.7484 - val_accuracy: 0.5000


In [ ]:
def predict(file):
    x = load_img(file, target_size=(img_width, img_height))
    x = img_to_array(x)
    x = np.expand_dims(x, axis=0)
    array = model.predict(x)
    result = array[0]
    if result[0] > result[1]:
        print("Predicted answer: Normal")
        answer = 'normal'
    else:
        print("Predicted answer: Tumor")
        answer = 'tumor'
    return answer


In [ ]:
tp = 0
tn = 0
fp = 0
fn = 0

for i, ret in enumerate(os.walk('./sample_data/test/tumor')):
    for i, filename in enumerate(ret[2]):
        if filename.startswith("."):
            continue
        print("Label: Tumor")
        result = predict(ret[0] + '/' + filename)
        if result == "tumor":
            tn += 1
        else:
            fp += 1

for i, ret in enumerate(os.walk('./sample_data/test/normal')):
    for i, filename in enumerate(ret[2]):
        if filename.startswith("."):
            continue
        print("Label: Normal")
        result = predict(ret[0] + '/' + filename)
        if result == "normal":
            tp += 1
        else:
            fn += 1


Label: Tumor
1/1 [==============================] - 0s 109ms/step
Predicted answer: Tumor
Label: Normal
1/1 [==============================] - 0s 26ms/step
Predicted answer: Tumor


In [ ]:
"""
Check metrics
"""
print("True Positive: ", tp)
print("True Negative: ", tn)
print("False Positive: ", fp)
print("False Negative: ", fn)

if (tp + fp) != 0:
    precision = tp / (tp + fp)
    print("Precision: ", precision)
else:
    print("Precision cannot be calculated because (tp+fp)=0")
    print("F-measure also cannot be calculated")
if (tp + fn) != 0:
    recall = tp / (tp + fn)
    print("Recall: ", recall)
else:
    print("Recall cannot be calculated because (tp+fn)=0")
    print("F-measure also cannot be calculated")
if (tp + fp) != 0 and (tp + fn) != 0:
    f_measure = (2 * recall * precision) / (recall + precision)
    print("F-measure: ", f_measure)

True Positive:  0
True Negative:  1
False Positive:  0
False Negative:  1
Precision cannot be calculated because (tp+fp)=0
F-measure also cannot be calculated
Recall:  0.0
